In [1]:
# idx=0
# irfs_opts, irfs, irfs_label, location = IRFS_OPTS[idx], IRFS[idx], IRFS_LABELS[idx], LOCATION[idx]

# irfs_opt = ['North', 'AverageAz', '20deg', '50h']
# irfs = Irfs.get_irfs(irfs_opt)
# irfs_label = Irfs.get_irfs_label(irfs_opt)
# location = Irfs.get_obs_loc(irfs_opt)

## CTA sensitivity for a point-like IRF 

[gammapy-tutorial](https://docs.gammapy.org/1.2/tutorials/analysis-1d/cta_sensitivity.html)</br>

[CTAO's expected "Alpha Configuration" performance](https://www.cta-observatory.org/science/ctao-performance/)

[CTAO Instrument Response Functions - prod5 version v0.1](https://zenodo.org/records/5499840#.YUya5WYzbUI)

[IRFs - gamma-astro-data-formats](https://gamma-astro-data-formats.readthedocs.io/en/v0.3/irfs/index.html)

In [2]:
%matplotlib inline

In [35]:
import matplotlib.pyplot as plt

import numpy as np
import astropy.units as u

from astropy.coordinates import Angle, SkyCoord

from feupy.utils.string_handling import name_to_txt
from feupy.utils.io import mkdir_sub_directory
from feupy.utils.table import write_tables_fits, write_tables_csv
from feupy.utils.datasets import flux_points_dataset_from_table

from feupy.utils.geometry import (
    create_energy_axis, 
    create_pointing, 
    create_pointing_position, 
    create_region_geometry,
    define_on_region,
)

from feupy.analysis.config import AnalysisConfig
from feupy.analysis.core import Analysis
from feupy.utils.observation import get_obs_label
from feupy.cta.irfs import Irfs

from gammapy.maps import MapAxis

from gammapy.datasets import SpectrumDataset, SpectrumDatasetOnOff, Datasets
from gammapy.makers import SpectrumDatasetMaker, SafeMaskMaker
from gammapy.estimators import FluxPoints, SensitivityEstimator

from gammapy.data import Observation

In [36]:
from core import irfs_label_txt

In [37]:
# irfs_groups =[
#     ['South', 'South-SSTSubArray','South-MSTSubArray','North','North-MSTSubArray', 'North-LSTSubArray'], 
#     ['AverageAz', 'SouthAz', 'NorthAz'], 
#     ['20deg','40deg','60deg'], 
#     ['0.5h', '5h', '50h']
# ]
# IRFS_OPTS, IRFS, IRFS_LABELS, LOCATION = Irfs.get_irf_groups(irfs_groups)

In [38]:
config = AnalysisConfig()

In [39]:
source_name = 'NGC 1275'

position = SkyCoord.from_name(source_name)
print(f'{position.ra.deg:.2f}, {position.dec.deg:.2f}')
print(f'{position.galactic.l.deg:.2f}, {position.galactic.b.deg:.2f}')

49.95, 41.51
150.58, -13.26


In [ ]:
path = Path(".")

In [44]:
from pathlib import Path
outdir = f"./{name_to_txt(source_name)}"

datasets_file = Path(outdir) / "datasets/datasets.yaml"
datasets_file.parent.mkdir(exist_ok=True)


In [49]:
filename = Path(outdir) / "sensitivity/sensitivity.fits"
filename.parent.mkdir(exist_ok=True)
filename = Path(outdir) / "data/data.dat"
filename.parent.mkdir(exist_ok=True)

In [20]:
outdir = f"./{name_to_txt(source_name)}"
outdir_path = mkdir_sub_directory(outdir)

datasets_path = mkdir_sub_directory(outdir, 'datasets')[1]
figures_path = mkdir_sub_directory(outdir, 'figures')[1]
sensitivity_path = mkdir_sub_directory(outdir, 'sensitivity')[1]
data_path = mkdir_sub_directory(outdir, 'data')[1]

config.general.outdir = outdir
config.general.datasets_file = f'{datasets_path}/datasets.yaml'
config.general.models_file = f'{datasets_path}/models.yaml'

Directory 'NGC_1275' created
Directory 'NGC_1275/datasets' created
Directory 'NGC_1275/figures' created
Directory 'NGC_1275/sensitivity' created
Directory 'NGC_1275/data' created


In [21]:
analysis = Analysis(config)
analysis.read_datasets()

Setting logging config: {'level': 'INFO', 'filename': None, 'filemode': None, 'format': None, 'datefmt': None}
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model s

No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.
No reference model set for FluxMaps. Assuming point 

In [28]:
pointing_angle = 0*u.deg
offset = 0.5*u.deg

on_region_radius = Angle("0.11 deg")

gamma_min = 10
n_sigma = 5 
bkg_syst_fraction = 0.10

selection = ["edisp", "background", "exposure"]

containment = 0.68

acceptance = 1
acceptance_off = 20

In [29]:
pointing_position = create_pointing_position(position, pointing_angle, offset)
pointing = create_pointing(pointing_position)
print(f"{pointing}\n")

on_region = define_on_region(center=position, radius=on_region_radius)
print(f"{on_region}\n")

e_edges_min = 1.0e-01*u.TeV
e_edges_max = 3.2e+01*u.TeV
nbin_edges = 12
config.datasets.geom.axes.energy.min = e_edges_min
config.datasets.geom.axes.energy.max = e_edges_max
config.datasets.geom.axes.energy.nbins = nbin_edges
config.datasets.geom.axes.energy.name = 'energy'
energy_settings = config.datasets.geom.axes.energy

e_edges_min = 3.2e-02*u.TeV
e_edges_max = 1.0e+02*u.TeV
nbin_edges = 15
config.datasets.geom.axes.energy_true.min =  e_edges_min
config.datasets.geom.axes.energy_true.max = e_edges_max
config.datasets.geom.axes.energy_true.nbins = nbin_edges
config.datasets.geom.axes.energy_true.name = 'energy_true'
energy_true_settings = config.datasets.geom.axes.energy_true

energy_axis = create_energy_axis(
    energy_settings.min, 
    energy_settings.max, 
    energy_settings.nbins, 
    per_decade=True, 
    name=energy_settings.name
)

geom = create_region_geometry(on_region, axes=[energy_axis])
print(geom)
print(energy_axis)

energy_axis_true = create_energy_axis(
    energy_true_settings.min, 
    energy_true_settings.max, 
    energy_true_settings.nbins, 
    per_decade=True, 
    name=energy_true_settings.name
)


empty_dataset = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true, name='empty_dataset')
# analysis.datasets = Datasets(empty_dataset)
print(empty_dataset)

FixedPointingInfo:

mode:        PointingMode.POINTING
coordinates: <SkyCoord (ICRS): (ra, dec) in deg
    (49.95066663, 42.0116969)>

Region: CircleSkyRegion
center: <SkyCoord (ICRS): (ra, dec) in deg
    (49.95066663, 41.5116969)>
radius: 0.11 deg

RegionGeom

	region     : CircleSkyRegion
	axes       : ['lon', 'lat', 'energy']
	shape      : (1, 1, 31)
	ndim       : 3
	frame      : icrs
	center     : 50.0 deg, 41.5 deg

MapAxis

	name       : energy    
	unit       : 'TeV'     
	nbins      : 31        
	node type  : edges     
	edges min  : 1.0e-01 TeV
	edges max  : 3.2e+01 TeV
	interp     : log       

SpectrumDataset
---------------

  Name                            : empty_dataset 

  Total counts                    : 0 
  Total background counts         : 0.00
  Total excess counts             : 0.00

  Predicted counts                : 0.00
  Predicted background counts     : 0.00
  Predicted excess counts         : nan

  Exposure min                    : 0.00e+00 m2 s
  Expos

In [30]:
spectrum_maker = SpectrumDatasetMaker(selection=selection)
sensitivity_estimator = SensitivityEstimator(
    gamma_min=gamma_min, n_sigma=n_sigma, bkg_syst_fraction=bkg_syst_fraction
)
sensitivity_estimator1 = SensitivityEstimator(
    gamma_min=gamma_min, n_sigma=n_sigma, bkg_syst_fraction=bkg_syst_fraction
)

In [31]:
livetime = 50 * u.h

In [32]:
irfs_config =[
    ['South', 'South-SSTSubArray','South-MSTSubArray','North','North-MSTSubArray', 'North-LSTSubArray'], 
    ['AverageAz'], 
    ['20deg','40deg','60deg'], 
    ['50h']
]
irfs_opts, irfss, irfs_labels, locations = Irfs.get_irf_groups(irfs_config)

In [33]:
datasets = Datasets()
for i, (irfs_opt, irfs, irfs_label, location) in enumerate(zip(irfs_opts, irfss, irfs_labels, locations)):
    obs = Observation.create(
        pointing=pointing, irfs=irfs, livetime=livetime, location=location
    )
    print(obs)
    obs_label = get_obs_label(irfs_opt, offset, on_region_radius, livetime)
    print(obs_label)
    dataset = spectrum_maker.run(empty_dataset, obs)
    print(dataset)

    # correct exposure
    print('correct exposure')
    dataset.exposure *= containment
    print(dataset)

    # correct background estimation
    print('correct background estimation')
    on_radii = obs.psf.containment_radius(
        energy_true=energy_axis.center, 
        offset=offset, 
        fraction=containment
    )
    factor = (1 - np.cos(on_radii)) / (1 - np.cos(geom.region.radius))
    dataset.background *= factor.value.reshape((-1, 1, 1))
    print(dataset)

    print('create SpectrumDatasetOnOff')
    dataset_on_off = SpectrumDatasetOnOff.from_spectrum_dataset(
        dataset=dataset, acceptance=acceptance, acceptance_off=acceptance_off
    )
    print(dataset_on_off)

    sensitivity_table = sensitivity_estimator.run(dataset_on_off)
    sensitivity_table.meta['source'] = source_name
    sensitivity_table.meta["offset"] = offset.to_string()
    sensitivity_table.meta["on_region_radius"] = f'{on_region_radius.deg}deg'
    sensitivity_table.meta["livetime"] = livetime.to_string()    
    sensitivity_table.meta["site"] = irfs_opt[0]
    sensitivity_table.meta["azimuth-averaged "] = irfs_opt[1]
    sensitivity_table.meta["zenith-angle"] = u.Quantity(irfs_opt[2]).to_string()
    sensitivity_table.meta["obs_time"] = u.Quantity(irfs_opt[3]).to_string()
    sensitivity_table.meta['irfs_label'] = irfs_label
    sensitivity_table.meta['irfs_config'] = irfs_opt
    sensitivity_table["on_radii"] = on_radii
    sensitivity_table["on_radii"].format = '.3e'
    label_table = f'sens-{irfs_label_txt(irfs_opt)}'
    label_dataset = f'sens {obs_label}'
    sensitivity_dataset = flux_points_dataset_from_table(sensitivity_table, name=label_dataset)
    datasets.append(sensitivity_dataset)
    print(sensitivity_table)
    print(dataset_on_off)

    dataset_on_off1 = dataset_on_off.to_image()

    sensitivity_table1 = sensitivity_estimator1.run(dataset_on_off1)
    print(sensitivity_table1)

    # To get the integral flux, we convert to a `FluxPoints` object that does the conversion
    # internally
    flux_points = FluxPoints.from_table(
        sensitivity_table1, sed_type="e2dnde", reference_model=sensitivity_estimator1.spectrum
    )
    int_sens = np.squeeze(flux_points.flux.quantity)
    print(
        f"Integral sensitivity in {livetime:.2f} above {e_edges_min:.2e} "
        f"is {int_sens:.2e}"
    )
    sensitivity_table.meta['int_sens'] = int_sens.to_string()
    write_tables_csv(
        sensitivity_table, sensitivity_path, label_table)

Observation

	obs id            : 0 
 	tstart            : 51544.00
	tstop             : 51546.08
	duration          : 180000.00 s
	pointing (icrs)   : 50.0 deg, 42.0 deg

	deadtime fraction : 0.0%

S-Az-20deg-50h (0.11deg,0.5deg,50h)


Table column name energy will be deprecated by e_ref since v1.2
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.


SpectrumDataset
---------------

  Name                            : empty_dataset 

  Total counts                    : 0 
  Total background counts         : 7174.32
  Total excess counts             : -7174.32

  Predicted counts                : 7174.32
  Predicted background counts     : 7174.32
  Predicted excess counts         : nan

  Exposure min                    : 3.97e+08 m2 s
  Exposure max                    : 5.51e+11 m2 s

  Number of total bins            : 31 
  Number of fit bins              : 31 

  Fit statistic type              : cash
  Fit statistic value (-2 log(L)) : nan

  Number of models                : 0 
  Number of parameters            : 0
  Number of free parameters       : 0


correct exposure
SpectrumDataset
---------------

  Name                            : empty_dataset 

  Total counts                    : 0 
  Total background counts         : 7174.32
  Total excess counts             : -7174.32

  Predicted counts                : 7174.32
 

Table column name energy will be deprecated by e_ref since v1.2


 energy  e_ref  e_min e_max     e2dnde     excess background criterion
  TeV     TeV    TeV   TeV  erg / (s cm2)                             
------- ------- ----- ----- ------------- ------- ---------- ---------
1.78885 1.78885   0.1    32   3.15016e-13 872.231    8722.31       bkg
Integral sensitivity in 50.00 h above 3.20e-02 TeV is 1.96e-12 1 / (s cm2)
Observation

	obs id            : 0 
 	tstart            : 51544.00
	tstop             : 51546.08
	duration          : 180000.00 s
	pointing (icrs)   : 50.0 deg, 42.0 deg

	deadtime fraction : 0.0%

S-Az-40deg-50h (0.11deg,0.5deg,50h)


...

 energy  e_ref  e_min e_max     e2dnde     excess background criterion
  TeV     TeV    TeV   TeV  erg / (s cm2)                             
------- ------- ----- ----- ------------- ------- ---------- ---------
1.78885 1.78885   0.1    32   4.86168e-13 684.007    6840.07       bkg
Integral sensitivity in 50.00 h above 3.20e-02 TeV is 3.02e-12 1 / (s cm2)
Observation

	obs id            : 0 
 	tstart            : 51544.00
	tstop             : 51546.08
	duration          : 180000.00 s
	pointing (icrs)   : 50.0 deg, 42.0 deg

	deadtime fraction : 0.0%

N-LST-Az-60deg-50h (0.11deg,0.5deg,50h)


Table column name energy will be deprecated by e_ref since v1.2
No reference model set for FluxMaps. Assuming point source with E^-2 spectrum.


SpectrumDataset
---------------

  Name                            : empty_dataset 

  Total counts                    : 0 
  Total background counts         : 13622.52
  Total excess counts             : -13622.52

  Predicted counts                : 13622.52
  Predicted background counts     : 13622.52
  Predicted excess counts         : nan

  Exposure min                    : 7.90e+06 m2 s
  Exposure max                    : 1.76e+11 m2 s

  Number of total bins            : 31 
  Number of fit bins              : 31 

  Fit statistic type              : cash
  Fit statistic value (-2 log(L)) : nan

  Number of models                : 0 
  Number of parameters            : 0
  Number of free parameters       : 0


correct exposure
SpectrumDataset
---------------

  Name                            : empty_dataset 

  Total counts                    : 0 
  Total background counts         : 13622.52
  Total excess counts             : -13622.52

  Predicted counts                : 136

Table column name energy will be deprecated by e_ref since v1.2


 energy  e_ref  e_min e_max     e2dnde    excess background criterion
  TeV     TeV    TeV   TeV  erg / (s cm2)                            
------- ------- ----- ----- ------------- ------ ---------- ---------
1.78885 1.78885   0.1    32   1.07108e-12   2347      23470       bkg
Integral sensitivity in 50.00 h above 3.20e-02 TeV is 6.66e-12 1 / (s cm2)


In [67]:
datasets.write(f'{datasets_path}/sensitivity_datasets.yaml', overwrite=True)

In [65]:
datasets_path

PosixPath('NGC_1275/datasets')

In [60]:
datasets.extend(analysis.datasets)

In [62]:
analysis.datasets = datasets

In [63]:
analysis.write_datasets()

Datasets stored to NGC_1275/datasets/datasets.yaml.
Datasets stored to NGC_1275/datasets/models.yaml.
